In [1]:
#libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt
import joypy
import requests
import json

import re

In [2]:
#!pip install joypy

### Creating various functions for webscraping on OneMap API and calculating distance
As we intend to collect data of various amenities, we create a function for finding the latitude and longitude value of the location of that amenity. Thereafter, we can compute the distance to the location of the HDB flat.

The API will compute and return the 'search value', latitude and longitude based on the given information such as building name, address or postal code.
'Search value' returned can be name of building or address based on input.

In [3]:
import requests
import json


# Function for getting postal code, geo coordinates of addresses

def find_postal(lst, filename):
    '''With the block number and street name, get the full address of the hdb flat,
    including the postal code, geogaphical coordinates (lat/long)'''
    
    for index,add in enumerate(lst):
        # URL on Onemap to retrieve 'Search Value'
        url= "https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal="+ add        
        print(index,url)
        
        # Retrieve information from website
        response = requests.get(url)
        try:
            data = json.loads(response.text) 
        except ValueError:
            print('JSONDecodeError')
            pass
    
        temp_df = pd.DataFrame.from_dict(data["results"])
        # The "add" is the address that was used to search in the website
        temp_df["address"] = add
        
        # Create the file with the first row that is read in 
        if index == 0:
            file = temp_df
        else:
            file = file.append(temp_df)
    file.to_csv(filename + '.csv')

### Finding the nearest amenities to a HDB flat
Below are functions that compute and find amenities within 2km radius with the assistance of geodesic from Geopy.
Geodesic helps to calculate distance between 2 points (HDB flat and the location of amenities based on the given parameters).
The distance between 2 points are computed based on given latitude and longitude of both buildings location.

In [4]:
# Function for getting closest distance of each location from a list of amenities location

from geopy.distance import geodesic

def find_nearest(house, amenity, radius=2):
    """
    this function finds the nearest locations from the 2nd table from the 1st address
    Both are dataframes with a specific format:
        1st column: any string column ie addresses taken from the dataframe or csv file
        2nd column: latitude (float)
        3rd column: longitude (float)
    Column name doesn't matter.
    It also finds the number of amenities within the given radius (default=2)
    """
    results = {}
    # first column must be address
    for index,flat in enumerate(house.iloc[:,0]):
        
        # 1st column of hdb flat is latitude, 2nd column is longitude
        flat_loc = (house.iloc[index,1],house.iloc[index,2])
        flat_amenity = ['','',100,0]
        for ind, eachloc in enumerate(amenity.iloc[:,0]):
            
            # 1st column of amenity is latitude, 2nd column is longitude
            amenity_loc = (amenity.iloc[ind,1],amenity.iloc[ind,2])
            distance = geodesic(flat_loc,amenity_loc)
            distance = float(str(distance)[:-3]) # convert to float

            if distance <= radius:   # compute number of amenities in 2km radius
                flat_amenity[3] += 1

            if distance < flat_amenity[2]: # find nearest amenity
                flat_amenity[0] = flat
                flat_amenity[1] = eachloc
                flat_amenity[2] = distance

        results[flat] = flat_amenity
    return results

### Finding the distance between 2 points
Below function is created solely for calculating the distance between HDB flats and City Hall (benchmarked as a central location point in the Central Business District (CBD) which is one of the features to predict prices later).

In [5]:
def dist_from_location(house, location):
    """
    this function finds the distance of a location from the 1st address
    First is a dataframe with a specific format:
        1st column: any string column ie addresses taken from the dataframe or csv file
        2nd column: latitude (float)
        3rd column: longitude (float)
    Second is tuple with latitude and longitude of location
    """
    results = {}
    # first column must be address
    for index,flat in enumerate(house.iloc[:,0]):
        
        # 2nd column must be latitude, 3rd column must be longitude
        flat_loc = (house.iloc[index,1],house.iloc[index,2])
        flat_amenity = ['',100]
        distance = geodesic(flat_loc,location)
        distance = float(str(distance)[:-3]) # convert to float
        flat_amenity[0] = flat
        flat_amenity[1] = distance
        results[flat] = flat_amenity
    return results

### EDA on HDB flats
After studying the HDB all prices dataset, I will select only the address, latitude and longitude (which has already been computed as per given dataset provided). Hence, for HDB flat, I do not need to scrap for latitude and longitude via OneMap API. 

In [6]:
hdb = pd.read_csv('All-prices-1990-2021.csv')


In [7]:
hdb

month  year        town  town_dummy flat_type block  \
0       1990-01  1990  ANG MO KIO           2    4 ROOM   308   
1       1990-01  1990  ANG MO KIO           2    3 ROOM   308   
2       1990-01  1990  ANG MO KIO           2    3 ROOM   216   
3       1990-01  1990  ANG MO KIO           2    3 ROOM   308   
4       1990-01  1990  ANG MO KIO           2    4 ROOM   211   
...         ...   ...         ...         ...       ...   ...   
840913  2021-03  2021      YISHUN           5    3 ROOM   110   
840914  2021-03  2021      YISHUN           5    3 ROOM   712   
840915  2021-03  2021      YISHUN           5    2 ROOM  424B   
840916  2021-03  2021      YISHUN           5    2 ROOM   459   
840917  2021-03  2021      YISHUN           5    2 ROOM   459   

             street_name                         address  latitude  \
0       ANG MO KIO AVE 1  308 ANG MO KIO AVE 1 SINGAPORE  1.365485   
1       ANG MO KIO AVE 1  308 ANG MO KIO AVE 1 SINGAPORE  1.365485   
2       ANG MO KIO AVE 1  216 ANG MO KIO AVE 1 SINGAPORE  1.366272   
3       ANG MO KIO AVE 1  308 ANG MO KIO AVE 1 SINGAPORE  1.365485   
4       ANG MO KIO AVE 3  211 ANG MO KIO AVE 3 SINGAPORE  1.369226   
...                  ...                             ...       ...   
840913    YISHUN RING RD    110 YISHUN RING RD SINGAPORE  1.433340   
840914      YISHUN AVE 5      712 YISHUN AVE 5 SINGAPORE  1.430248   
840915     YISHUN AVE 11    424B YISHUN AVE 11 SINGAPORE  1.423215   
840916     YISHUN AVE 11     459 YISHUN AVE 11 SINGAPORE  1.421059   
840917     YISHUN AVE 11     459 YISHUN AVE 11 SINGAPORE  1.421059   

         longitude  ... price_psm_yearly  Core CPI  price cpi_adj  \
0       103.844025  ...        10.997442     61.59   139633.05730   
1       103.844025  ...         8.464849     61.59    95794.77188   
2       103.841465  ...         7.606769     61.59    76635.81750   
3       103.844025  ...         8.287972     61.59    76635.81750   
4       103.841652  ...        11.484353     61.59   129891.21610   
...            ...  ...              ...       ...            ...   
840913  103.829168  ...        66.650391    100.40   271912.35060   
840914  103.828862  ...        63.567362    100.40   266932.27090   
840915  103.848264  ...        59.482956    100.40   258964.14340   
840916  103.846357  ...        57.234432    100.40   249003.98410   
840917  103.846357  ...        48.076923    100.40   209163.34660   

       price_psm cpi_adj  bala lease pct  price lease_adj implied  \
0            1517.750623            92.9              88869.75242   
1            1168.228925            92.9              60968.78364   
2            1049.805719            92.9              48775.02691   
3            1143.818172            92.9              48775.02691   
4            1603.595261            93.3              82315.11254   
...                  ...             ...                      ...   
840913       4248.630478            82.4             318058.25240   
840914       3925.474573            81.2             316847.29060   
840915       5509.875392            95.2             262184.87390   
840916       5187.583001            94.8             253164.55700   
840917       4357.569721            94.8             212658.22780   

        price_psm lease_adj implied  price cpi_lease_adj implied  \
0                        965.975570                 144292.50270   
1                        743.521752                  98991.36814   
2                        668.151054                  79193.09451   
3                        727.985476                  79193.09451   
4                       1016.235957                 133650.12590   
...                             ...                          ...   
840913                  4969.660194                 316791.08810   
840914                  4659.518980                 315584.95080   
840915                  5578.401574                 261140.31270   
840916                  5274.261603               

In [8]:
# Selecting only certain columns in order to retabulate address column to remove the word 'Singapore' to identify the Block number and street name as name of the HDB flat.
hdb = hdb[['block', 'street_name', 'latitude', 'longitude']]
hdb['address'] = hdb['block'] + ' ' + hdb['street_name']
all_address = list(hdb['address'])
unique_address = list(set(all_address))

# To find how many buildings of blocks of flats are in this data
print('Unique addresses:', len(unique_address))

Unique addresses: 9381


<ipython-input-8-e1b848e30e78>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hdb['address'] = hdb['block'] + ' ' + hdb['street_name']


In [9]:
hdb = hdb.sort_values('street_name', ascending = False).drop_duplicates('address').sort_index()

In [10]:
hdb

block        street_name  latitude   longitude                address
49       401  ANG MO KIO AVE 10  1.361100  103.855419  401 ANG MO KIO AVE 10
52       414  ANG MO KIO AVE 10  1.364074  103.853485  414 ANG MO KIO AVE 10
53       438  ANG MO KIO AVE 10  1.366973  103.853807  438 ANG MO KIO AVE 10
56       418  ANG MO KIO AVE 10  1.364935  103.854061  418 ANG MO KIO AVE 10
57       443  ANG MO KIO AVE 10  1.365705  103.856107  443 ANG MO KIO AVE 10
...      ...                ...       ...         ...                    ...
840739  780A     WOODLANDS CRES  1.448820  103.800668    780A WOODLANDS CRES
840740  782C     WOODLANDS CRES  1.447799  103.803439    782C WOODLANDS CRES
840755    19     MARSILING LANE  1.443162  103.777238      19 MARSILING LANE
840812  511A       YISHUN ST 51  1.415666  103.841400      511A YISHUN ST 51
840864  342B     YISHUN RING RD  1.430229  103.844785    342B YISHUN RING RD

[9381 rows x 5 columns]

In [11]:
# Removing other columns that are not required and are also duplicated info from address column
hdb.drop('block', inplace=True, axis=1)
hdb.drop('street_name', inplace=True, axis=1)

In [12]:
hdb

latitude   longitude                address
49      1.361100  103.855419  401 ANG MO KIO AVE 10
52      1.364074  103.853485  414 ANG MO KIO AVE 10
53      1.366973  103.853807  438 ANG MO KIO AVE 10
56      1.364935  103.854061  418 ANG MO KIO AVE 10
57      1.365705  103.856107  443 ANG MO KIO AVE 10
...          ...         ...                    ...
840739  1.448820  103.800668    780A WOODLANDS CRES
840740  1.447799  103.803439    782C WOODLANDS CRES
840755  1.443162  103.777238      19 MARSILING LANE
840812  1.415666  103.841400      511A YISHUN ST 51
840864  1.430229  103.844785    342B YISHUN RING RD

[9381 rows x 3 columns]

### For consistency in every dataframe 
For consistency in every csv file created later, I am going to make every data file with 'Address' as index column, then latitude and longitude as per required format in 'find_nearest' and 'distance_from_location' method created above.

In [13]:
# 'Popping' address column to reassign as index column
col_name="address"
first_col = hdb.pop(col_name)

In [14]:
hdb.head()

latitude   longitude
49  1.361100  103.855419
52  1.364074  103.853485
53  1.366973  103.853807
56  1.364935  103.854061
57  1.365705  103.856107

In [15]:
hdb.insert(0, col_name, first_col)

In [16]:
hdb

address  latitude   longitude
49      401 ANG MO KIO AVE 10  1.361100  103.855419
52      414 ANG MO KIO AVE 10  1.364074  103.853485
53      438 ANG MO KIO AVE 10  1.366973  103.853807
56      418 ANG MO KIO AVE 10  1.364935  103.854061
57      443 ANG MO KIO AVE 10  1.365705  103.856107
...                       ...       ...         ...
840739    780A WOODLANDS CRES  1.448820  103.800668
840740    782C WOODLANDS CRES  1.447799  103.803439
840755      19 MARSILING LANE  1.443162  103.777238
840812      511A YISHUN ST 51  1.415666  103.841400
840864    342B YISHUN RING RD  1.430229  103.844785

[9381 rows x 3 columns]

In [17]:
#Saving file to a csv for use in main ipynb (HDB Main.ipynb) file later 
hdb.to_csv(r'flat_coordinates.csv')

### Scraping for location data of supermarket

Below file 'list-of-supermarket-licences.csv' is taken from datagov.
https://data.gov.sg/dataset/list-of-supermarket-licences

We then use the postal code to get the latitude and longitude information of the location of the supermarket.
This information then can be used to compute the distance and number of supermarket within 2km radius when applying the function 'find_nearest' and 'distance_from_location' created above.


In [18]:
# Reading file and convert to a dataframe
supermarket = pd.read_csv('list-of-supermarket-licences.csv')
supermarket.head()

licence_num                          licensee_name  \
0   S02185J000  COLD STORAGE SINGAPORE (1983) PTE LTD   
1   S97166P000  COLD STORAGE SINGAPORE (1983) PTE LTD   
2  CE15B61K000          U STARS SUPERMARKET PTE. LTD.   
3   S02029J000        NTUC Fairprice Co-operative Ltd   
4  CE04561V000        SHENG SIONG SUPERMARKET PTE LTD   

                 building_name block_house_num level_num unit_num  \
0                           na             982         1        1   
1  HOUGANG GREEN SHOPPING MALL              21         1       52   
2                           na             330         1        1   
3                HOUGANG POINT               1         2        1   
4                           na              19         1    42401   

                street_name postal_code  
0         BUANGKOK CRESCENT      530982  
1         HOUGANG STREET 51      538719  
2         ANCHORVALE STREET      540330  
3         HOUGANG STREET 91      538692  
4  SERANGOON NORTH AVENUE 5      554913

In [19]:
# Selecting only the postal_code column for scrapping via OneMap API
supermarket_address = list(supermarket['postal_code'])

# As we suspect that there may be duplicate data, we need to check for unique address based on postal code.
# Unique postal code shown below is 445, but when we get the number of rows for supermarket['postal_code'] it returns a value of 478.
# This shows that there are 33 duplicates.
unique_supermarket_address = list(set(supermarket_address))
print('Unique addresses:', len(unique_supermarket_address))

Unique addresses: 445


In [20]:
supermarket['postal_code']

0      530982
1      538719
2      540330
3      538692
4      554913
        ...  
473    127371
474    600061
475    600037
476    127371
477    129787
Name: postal_code, Length: 478, dtype: object

In [21]:
# Applying the find_postal function which as per mentioned above, scrap for latitude and logitude values 
# based on input, in this case - postal code of supermarket.
# The file will be created as supermarket_coordinates.csv
find_postal(unique_supermarket_address, 'supermarket_coordinates')

0 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=637397
1 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=650280
2 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=415812
3 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=619415
4 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=079027
5 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=389344
6 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=552542
7 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=709155
8 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=739065
9 https://developers.onemap.sg/commonapi/search?returnG

80 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=269707
81 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=738252
82 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=757444
83 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=389290
84 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=270021
85 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=486442
86 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=228209
87 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=540263
88 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=415920
89 https://developers.onemap.sg/commonapi/sear

160 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=679947
161 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=168732
162 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=0na
163 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=651154
164 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=579837
165 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=769093
166 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=039594
167 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=129588
168 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=757443
169 https://developers.onemap.sg/commona

238 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=618642
239 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=469979
240 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=680142
241 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=307683
242 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=449269
243 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=380011
244 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=608586
245 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=531159
246 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=210043
247 https://developers.onemap.sg/comm

317 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=160055
318 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=389538
319 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=658713
320 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=650166
321 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=798049
322 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=597626
323 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=791447
324 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=339511
325 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=358361
326 https://developers.onemap.sg/comm

397 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=820301
398 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=200808
399 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=737723
400 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=638127
401 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=560122
402 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=659003
403 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=520823
404 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=636937
405 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=760845
406 https://developers.onemap.sg/comm

In [22]:
# Computed values from OneMap API is then tabulated into a dataframe and saved as supermarket_coordinates.csv
# With the latitude and longitude information, we can calculate distance and find the number of supermarkets within 2km of every HDB flat.

supermarket = pd.read_csv('supermarket_coordinates.csv')
supermarket.drop_duplicates(subset=['address'], inplace=True)
supermarket = supermarket[['address','LATITUDE', 'LONGTITUDE']]
supermarket

address  LATITUDE  LONGTITUDE
0      650280  1.347701  103.757045
3      415812  1.337247  103.903853
4      619415  1.321281  103.729833
6       79027  1.274588  103.843471
9      389344  1.312724  103.878705
...       ...       ...         ...
1389   729939  1.407543  103.758095
1390   760780  1.422139  103.832883
1391   510259  1.366454  103.964598
1394   259760  1.322573  103.814246
1395   510258  1.367001  103.964618

[429 rows x 3 columns]

In [23]:
# Applying the find_nearest() function to compute the nearest supermarket to flat and find the number of supermarkets within 2km radius.
nearest_supermarket = find_nearest(hdb, supermarket)
flat_supermarket = pd.DataFrame.from_dict(nearest_supermarket).T

# Passing the values after applying the find_nearest function into a dataframe with columns as shown below, in such order.
flat_supermarket = flat_supermarket.rename(columns={0: 'flat', 1: 'supermarket_name', 2: 'supermarket_dist', 3: 'num_supermarket_2km'}).reset_index().drop('index', axis=1)
flat_supermarket.head()

flat supermarket_name supermarket_dist num_supermarket_2km
0  401 ANG MO KIO AVE 10           560421         0.546651                  17
1  414 ANG MO KIO AVE 10           560421         0.155975                  17
2  438 ANG MO KIO AVE 10           560421          0.19899                  15
3  418 ANG MO KIO AVE 10           560421         0.131461                  17
4  443 ANG MO KIO AVE 10           560421         0.351198                  16

In [24]:
#Saving the dataframe into a csv file
flat_supermarket.to_csv(r'flat_supermarket.csv')

### Scraping for location data of shopping mall

Below file 'shopping_malls.csv' is personally tabulated from Wikipedia.
https://en.wikipedia.org/wiki/List_of_shopping_malls_in_Singapore

We then use the postal code to get the latitude and longitude information of the location of the supermarket.
This information then can be used to compute the distance and number of supermarket within 2km radius when applying the function 'find_nearest' and 'distance_from_location' created above.


In [25]:
# Reading file and convert to a dataframe
shopping_mall = pd.read_csv('shopping_malls.csv')
shopping_mall.head()

mall
0                          100 AM
1                    313@Somerset
2                          Aperia
3  Balestier Hill Shopping Centre
4                      Bugis Cube

In [26]:
# Finding out the number of malls based on number of rows (len function)
shopping_mall_name = list(shopping_mall['mall'])
print('No. of malls:', len(shopping_mall_name))

No. of malls: 171


In [27]:
shopping_mall_name

['100 AM',
 '313@Somerset',
 'Aperia',
 'Balestier Hill Shopping Centre',
 'Bugis Cube',
 'Bugis Junction',
 'Bugis+',
 'Capitol Piazza',
 'Cathay Cineleisure Orchard',
 'Clarke Quay Central',
 'The Centrepoint',
 'City Square Mall',
 'City Gate Mall',
 'CityLink Mall',
 'Duo',
 'Far East Plaza',
 'Funan',
 'Great World City',
 'HDB Hub',
 'Holland Village Shopping Mall',
 'ION Orchard',
 'Junction 8',
 'Knightsbridge',
 'Liat Towers',
 'Lucky Plaza',
 'Marina Bay Sands',
 'The Shoppes at Marina Bay Sands',
 'Marina Bay Financial Centre Tower 3',
 'Marina Bay Link Mall',
 'Marina One',
 'Marina Square',
 'Millenia Walk',
 'Mustafa Shopping Centre',
 'Ngee Ann City',
 'Orchard Central',
 'Orchard Gateway',
 'Orchard Plaza',
 'Midpoint Orchard',
 'Palais Renaissance',
 "People's Park Centre",
 "People's Park Complex",
 'Plaza Singapura',
 'PoMo',
 'Raffles City',
 'Scotts Square',
 'Serangoon Plaza',
 'Shaw House and Centre',
 'Sim Lim Square',
 'Singapore Shopping Centre',
 'The South B

In [28]:
# Applying the find_postal function which as per mentioned above, scrap for latitude and logitude values 
# based on input, in this case - name of shopping mall.
find_postal(shopping_mall_name, 'shoppingmall_coordinates')

0 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=100 AM
1 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=313@Somerset
2 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Aperia
3 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Balestier Hill Shopping Centre
4 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Bugis Cube
5 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Bugis Junction
6 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Bugis+
7 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Capitol Piazza
8 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Cathay Cinel

75 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Bedok Mall
76 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Bedok Point
77 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Paya Lebar Square
78 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Singapore Post Centre (Singpost Centre)
79 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Paya Lebar Quarter (PLQ)
80 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=KINEX (formerly OneKM)
81 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Katong Shopping Centre
82 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Roxy Square
83 https://developers.onemap.sg/commonapi/se

148 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=VivoCity
149 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=HarbourFront Centre
150 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Alexandra Retail Centre
151 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=321 Clementi
152 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=The Clementi Mall
153 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=IMM
154 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=JCube
155 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Jem
156 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=We

In [29]:
# The file will be created as shoppingmall_coordinates.csv
shopping_mall = pd.read_csv('shoppingmall_coordinates.csv')
shopping_mall = shopping_mall[['address','LATITUDE', 'LONGTITUDE']]
shopping_mall.head()

address  LATITUDE  LONGTITUDE
0        100 AM  1.274588  103.843471
1  313@Somerset  1.301385  103.837684
2  313@Somerset  1.301007  103.838247
3        Aperia  1.309742  103.864102
4        Aperia  1.309711  103.864326

In [30]:
# Finding out the number of malls based on number of rows (len function), to confirm is the numbers tally, or more rows or less rows of data was scraped.
print('No. of malls:', len(shopping_mall))

No. of malls: 649


In [31]:
# Dropping duplicate data based on name under the column 'address'
shopping_mall = shopping_mall.drop_duplicates(subset = ["address"])
print('No. of malls:', len(shopping_mall))

No. of malls: 158


In [32]:
shopping_mall

address  LATITUDE  LONGTITUDE
0                            100 AM  1.274588  103.843471
1                      313@Somerset  1.301385  103.837684
3                            Aperia  1.309742  103.864102
8    Balestier Hill Shopping Centre  1.325761  103.843054
12                       Bugis Cube  1.298195  103.855656
..                              ...       ...         ...
630         Gek Poh Shopping Centre  1.348744  103.698299
634                  Rochester Mall  1.305408  103.788447
635    Taman Jurong Shopping Centre  1.334845  103.720462
641                West Coast Plaza  1.303743  103.766093
646       Queensway Shopping Centre  1.287618  103.803391

[158 rows x 3 columns]

In [33]:
# Applying the find_nearest() function to compute the nearest supermarket to flat and find the number of supermarkets within 2km radius.
# Passing the values after applying the find_nearest function into a dataframe with columns as shown below, in such order.
nearest_mall = find_nearest(hdb, shopping_mall)
flat_mall = pd.DataFrame.from_dict(nearest_mall).T
flat_mall = flat_mall.rename(columns={0: 'flat', 1: 'mall', 2: 'mall_dist', 3: 'num_mall_2km'}).reset_index().drop('index', axis=1)
flat_mall.head()

flat                           mall mall_dist num_mall_2km
0  401 ANG MO KIO AVE 10  myVillage At Serangoon Garden  1.161261            5
1  414 ANG MO KIO AVE 10                        AMK Hub   0.80033            5
2  438 ANG MO KIO AVE 10                        AMK Hub   0.64598            5
3  418 ANG MO KIO AVE 10                        AMK Hub   0.78492            5
4  443 ANG MO KIO AVE 10                        AMK Hub  0.936753            5

In [34]:
#Saving the dataframe into a csv file
flat_mall.to_csv(r'flat_mall.csv')

### Scraping for MRT stations

Below file 'train-station-chinese-names.csv' is taken from Data.gov website below.
https://data.gov.sg/dataset/train-station-chinese-names

We then use the postal code to get the latitude and longitude information of the location of the supermarket.
This information then can be used to compute the distance and number of MRT stations within 2km radius when applying the function 'find_nearest' and 'distance_from_location' created above.

In [35]:
# Reading file and convert to a dataframe
mrt = pd.read_csv('train-station-chinese-names.csv')
mrt.head()

stn_code mrt_station_english  mrt_line_english
0      NS1         Jurong East  North South Line
1      NS2         Bukit Batok  North South Line
2      NS3        Bukit Gombak  North South Line
3      NS4       Choa Chu Kang  North South Line
4      NS5             Yew Tee  North South Line

### To prevent unwanted buildings (not MRT station longitude and latitude values)

If we only apply mrt_station_english, we may find every other elements that has the same name in Singapore such as a building name called 'Ang Mo Kio'. To prevent this, we include the station code as it is embedded in the OneMap API data to have station name info as such. This way, we only get the right latitude and longitude for every MRT station.


In [36]:
#Creating a new column in dataframe called 'name' which includes the station name and code
mrt['name'] = mrt['mrt_station_english'] + ' ' + mrt['stn_code']
all_mrt = list(mrt['name'])
unique_mrt = list(set(all_mrt))

#Print out the number of MRT stations in the list
print('Number of MRT stations:', len(unique_mrt))

Number of MRT stations: 184


In [37]:
all_mrt

['Jurong East NS1',
 'Bukit Batok NS2',
 'Bukit Gombak NS3',
 'Choa Chu Kang NS4',
 'Yew Tee NS5',
 'Kranji NS7',
 'Marsiling NS8',
 'Woodlands NS9',
 'Admiralty NS10',
 'Sembawang NS11',
 'Yishun NS13',
 'Khatib NS14',
 'Yio Chu Kang NS15',
 'Ang Mo Kio NS16',
 'Bishan NS17',
 'Braddell NS18',
 'Toa Payoh NS19',
 'Novena NS20',
 'Newton NS21',
 'Orchard NS22',
 'Somerset NS23',
 'Dhoby Ghaut NS24',
 'City Hall NS25',
 'Raffles Place NS26',
 'Marina Bay NS27',
 'Marina South Pier NS28',
 'Pasir Ris EW1',
 'Tampines EW2',
 'Simei EW3',
 'Tanah Merah EW4',
 'Bedok EW5',
 'Kembangan EW6',
 'Eunos EW7',
 'Paya Lebar EW8',
 'Aljunied EW9',
 'Kallang EW10',
 'Lavender EW11',
 'Bugis EW12',
 'City Hall EW13',
 'Raffles Place EW14',
 'Tanjong Pagar EW15',
 'Outram Park EW16',
 'Tiong Bahru EW17',
 'Redhill EW18',
 'Queenstown EW19',
 'Commonwealth EW20',
 'Buona Vista EW21',
 'Dover EW22',
 'Clementi EW23',
 'Jurong East EW24',
 'Chinese Garden EW25',
 'Lakeside EW26',
 'Boon Lay EW27',
 'Pion

In [38]:
# Applying the find_postal function which as per mentioned above, scrap for latitude and logitude values 
# based on input, in this case - postal code of supermarket.
# The file will be created as mrt_coordinates.csv
find_postal(all_mrt, 'mrt_coordinates')

0 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Jurong East NS1
1 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Bukit Batok NS2
2 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Bukit Gombak NS3
3 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Choa Chu Kang NS4
4 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Yew Tee NS5
5 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Kranji NS7
6 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Marsiling NS8
7 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Woodlands NS9
8 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=

74 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Buangkok NE15
75 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Sengkang NE16
76 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Punggol NE17
77 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Dhoby Ghaut CC1
78 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Bras Basah CC2
79 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Esplanade CC3
80 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Promenade CC4
81 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Nicoll Highway CC5
82 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&

148 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Pending BP8
149 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Bangkit BP9
150 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Fajar BP10
151 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Segar BP11
152 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Jelapang BP12
153 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Senja BP13
154 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Ten Mile Junction BP14
155 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Sengkang STC
156 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&sea

In [39]:
# Computed values from OneMap API is then tabulated into a dataframe and saved as MRT_coordinates.csv
# With the latitude and longitude information, we can calculate distance and find the number of MRT stations within 2km of every HDB flat.
MRT = pd.read_csv('mrt_coordinates.csv')
MRT = MRT[['address','LATITUDE', 'LONGTITUDE']]
MRT.head()

address  LATITUDE  LONGTITUDE
0    Jurong East NS1  1.333153  103.742286
1    Bukit Batok NS2  1.349033  103.749566
2   Bukit Gombak NS3  1.358612  103.751791
3  Choa Chu Kang NS4  1.385363  103.744371
4        Yew Tee NS5  1.397535  103.747405

In [40]:
# Finding out the number of MRT stations based on number of rows (len function)
# To ensure that One Map API really have given us the latitude and longitude for ALL MRT stations based on the information passed.
print('No. of MRT:', len(MRT))
MRT

No. of MRT: 183


address  LATITUDE  LONGTITUDE
0      Jurong East NS1  1.333153  103.742286
1      Bukit Batok NS2  1.349033  103.749566
2     Bukit Gombak NS3  1.358612  103.751791
3    Choa Chu Kang NS4  1.385363  103.744371
4          Yew Tee NS5  1.397535  103.747405
..                 ...       ...         ...
178  Punggol Point PW3  1.416849  103.906651
179       Samudera PW4  1.415902  103.902156
180         Nibong PW5  1.411870  103.900314
181         Sumang PW6  1.408452  103.898558
182       Soo Teck PW7  1.405323  103.897291

[183 rows x 3 columns]

In [41]:
# Applying the find_nearest() function to compute the nearest MRT to flat and find the number of supermarkets within 2km radius.
# Passing the values after applying the find_nearest function into a dataframe with columns as shown below, in such order.
nearest_mrt = find_nearest(hdb, MRT)
flat_mrt = pd.DataFrame.from_dict(nearest_mrt).T
flat_mrt = flat_mrt.rename(columns={0: 'flat', 1: 'MRT', 2: 'MRT_dist', 3: 'num_MRT_2km'}).reset_index().drop('index', axis=1)
flat_mrt.head()

flat              MRT  MRT_dist num_MRT_2km
0  401 ANG MO KIO AVE 10  Ang Mo Kio NS16  1.174407           4
1  414 ANG MO KIO AVE 10  Ang Mo Kio NS16  0.781515           4
2  438 ANG MO KIO AVE 10  Ang Mo Kio NS16  0.575116           4
3  418 ANG MO KIO AVE 10  Ang Mo Kio NS16  0.745965           4
4  443 ANG MO KIO AVE 10  Ang Mo Kio NS16  0.865839           4

In [42]:
flat_mrt.to_csv(r'flat_mrt.csv')

### Applying only find_nearest function for hawker data
This is done and included in this notebook as I discovered the availability of a file on data.gov with latitude and longitude included for data on hawker centres in Singapore. Any future computation would only required to update hawker-centres-kml

Data can be found in website below:

https://data.gov.sg/dataset/hawker-centres

In [43]:
# Reading file and convert to a dataframe
hawker_data2 = pd.read_csv('hawker-centres-kml.csv')
hawker_data2.head()

Name           X         Y  \
0                              Ci Yuan Hawker Centre  103.882978  1.375429   
1       West Coast Drive Blk 502 (Ayer Rajah Market)  103.759166  1.311890   
2  West Coast Drive Blk 503 (Ayer Rajah Food Centre)  103.759751  1.311803   
3                            Bedok South Road Blk 16  103.935539  1.320552   
4                    Pasir Ris Central Hawker Centre  103.951543  1.373434   

              description ADDRESSBLOCKHOUSENUMBER  LATITUDE          STATUS  \
0              New Centre                      51  1.374938  Existing (new)   
1    Not eligible for HUP                     502  1.311960        Existing   
2  HUP Standard Upgrading                     503  1.311870        Existing   
3  HUP Standard Upgrading                      16  1.320560        Existing   
4              New Centre                     110  1.373318  Existing (new)   

  CLEANINGSTARTDATE  ADDRESSUNITNUMBER  ADDRESSFLOORNUMBER  ...  \
0          2/9/2019                NaN                 NaN  ...   
1         24/9/2019                NaN                 NaN  ...   
2         24/9/2019                NaN                 NaN  ...   
3         10/6/2019                NaN                 NaN  ...   
4         19/8/2019                NaN                 NaN  ...   

               ADDRESSBUILDINGNAME  COMPLETION_DATE LANDXADDRESSPOINT  \
0                              NaN             2015          33526.48   
1                              NaN              NaN          19747.53   
2                              NaN       15/12/2002          19812.66   
3                              NaN        20/9/2012          39376.14   
4  Pasir Ris Central Hawker Centre             2017          41156.94   

   ADDRESSSTREETNAME ADDRESSPOSTALCODE  \
0   Hougang Avenue 9          538776.0   
1   West Coast Drive          120502.0   
2   West Coast Drive          120503.0   
3   Bedok South Road          460016.0   
4  Pasir Ris Central          519641.0   

                                   DESCRIPTION_MYENV IMPLEMENTATION_DATE  \
0  Built in 2015, Ci Yuan hawker centre is the fi...                 NaN   
1  Also known as Ayer Rajah Market, the standalon...                 NaN   
2  Also known as Ayer Rajah Food Centre, the stan...                 NaN   
3  Located near schools, factories and housing es...                 NaN   
4  Completed in Jan 2018, Pasir Ris Central Hawke...           29/7/2015   

                                  ADDRESS_MYENV           INC_CRC  \
0         51 Hougang Avenue 9, Singapore 538776  2AA90E9C1246F8A3   
1   Blk 502, West Coast Drive, Singapore 120502  2DBB0A9FF06FDAA2   
2   Blk 503, West Coast Drive, Singapore 120503  7A2798344B6AD8F0   
3    Blk 16, Bedok South Road, Singapore 460016  483F3877E1D079FE   
4  Blk 110, Pasir Ris Central, Singapore 519641  0EC198F94A578B81   

     FMEL_UPD_D  
0  2.019080e+13  
1  2.019090e+13  
2  2.019090e+13  
3  2.019090e+13  
4  2.019070e+13  

[5 rows x 34 columns]

In [44]:
hawker_data2.drop(hawker_data2.columns.difference(['Name','X', 'Y']), 1, inplace=True)
hawker_data2

Name           X         Y
0                                Ci Yuan Hawker Centre  103.882978  1.375429
1         West Coast Drive Blk 502 (Ayer Rajah Market)  103.759166  1.311890
2    West Coast Drive Blk 503 (Ayer Rajah Food Centre)  103.759751  1.311803
3                              Bedok South Road Blk 16  103.935539  1.320552
4                      Pasir Ris Central Hawker Centre  103.951543  1.373434
..                                                 ...         ...       ...
115               Queen Street Blk 270 (Albert Centre)  103.854212  1.301001
116  Sembawang Hills Food Centre (Jalan Leban Food ...  103.828994  1.372385
117                                   Our Tampines Hub  103.939726  1.353862
118  Toa Payoh Lorong 4 Blk 74 (Toa Payoh Vista Mar...  103.851999  1.334548
119  Bukit Merah Lane 1 Blk 120 (Alexandra Village ...  103.804489  1.286298

[120 rows x 3 columns]

In [45]:
#Renaming column names
columns_titles = ['Name','Y', 'X']
hawker_data2=hawker_data2.reindex(columns=columns_titles)
hawker_data2

Name         Y           X
0                                Ci Yuan Hawker Centre  1.375429  103.882978
1         West Coast Drive Blk 502 (Ayer Rajah Market)  1.311890  103.759166
2    West Coast Drive Blk 503 (Ayer Rajah Food Centre)  1.311803  103.759751
3                              Bedok South Road Blk 16  1.320552  103.935539
4                      Pasir Ris Central Hawker Centre  1.373434  103.951543
..                                                 ...       ...         ...
115               Queen Street Blk 270 (Albert Centre)  1.301001  103.854212
116  Sembawang Hills Food Centre (Jalan Leban Food ...  1.372385  103.828994
117                                   Our Tampines Hub  1.353862  103.939726
118  Toa Payoh Lorong 4 Blk 74 (Toa Payoh Vista Mar...  1.334548  103.851999
119  Bukit Merah Lane 1 Blk 120 (Alexandra Village ...  1.286298  103.804489

[120 rows x 3 columns]

In [46]:
# Applying the find_nearest() function to compute the nearest hawker to flat and find the number of hawker within 2km radius.
# Passing the values after applying the find_nearest function into a dataframe with columns as shown below, in such order.
nearest_hawker = find_nearest(hdb, hawker_data2)
flat_hawker = pd.DataFrame.from_dict(nearest_hawker).T
flat_hawker = flat_hawker.rename(columns={0: 'flat', 1: 'hawker', 2: 'hawker_dist', 3: 'num_hawker_2km'}).reset_index().drop('index', axis=1)
flat_hawker.head()

flat                                             hawker  \
0  401 ANG MO KIO AVE 10       Ang Mo Kio Ave 10 Blk 409 (Teck Ghee Square)   
1  414 ANG MO KIO AVE 10       Ang Mo Kio Ave 10 Blk 409 (Teck Ghee Square)   
2  438 ANG MO KIO AVE 10  Ang Mo Kio Ave 10 Blk 453A (Chong Boon Market ...   
3  418 ANG MO KIO AVE 10       Ang Mo Kio Ave 10 Blk 409 (Teck Ghee Square)   
4  443 ANG MO KIO AVE 10  Ang Mo Kio Ave 10 Blk 453A (Chong Boon Market ...   

  hawker_dist num_hawker_2km  
0    0.184587              9  
1    0.254548             10  
2    0.315933             10  
3     0.28037             10  
4    0.281704              9

In [47]:
#Saving the dataframe into a csv file
flat_hawker.to_csv(r'flat_hawker.csv')

### Scraping for location of schools

Below file 'general-information-of-schools.csv' is taken from Data.gov website below.

https://data.gov.sg/dataset/school-directory-and-information

We then use the postal code to get the latitude and longitude information of the location of the supermarket.
This information then can be used to compute the distance and number of schools within 2km radius when applying the function 'find_nearest' and 'distance_from_location' created above.

In [48]:
# Reading file and convert to a dataframe
school_data = pd.read_csv('general-information-of-schools.csv')
school_data.head()

school_name                            url_address  \
0        ADMIRALTY PRIMARY SCHOOL       https://admiraltypri.moe.edu.sg/   
1      ADMIRALTY SECONDARY SCHOOL     http://www.admiraltysec.moe.edu.sg   
2    AHMAD IBRAHIM PRIMARY SCHOOL  http://www.ahmadibrahimpri.moe.edu.sg   
3  AHMAD IBRAHIM SECONDARY SCHOOL  http://www.ahmadibrahimsec.moe.edu.sg   
4                  AI TONG SCHOOL           http://www.aitong.moe.edu.sg   

                   address  postal_code telephone_no telephone_no_2    fax_no  \
0    11   WOODLANDS CIRCLE       738907     63620598             na  63627512   
1  31   WOODLANDS CRESCENT       737916     63651733       63654596  63652774   
2    10   YISHUN STREET 11       768643     67592906             na  67592927   
3     751  YISHUN AVENUE 7       768928     67585384             na  67557778   
4   100  Bright Hill Drive       579646     64547672             na  64532726   

  fax_no_2            email_address                  mrt_desc  ...  \
0       na  ADMIRALTY_PS@MOE.EDU.SG         Admiralty Station  ...   
1       na  Admiralty_SS@moe.edu.sg             ADMIRALTY MRT  ...   
2       na          aips@moe.edu.sg                    Yishun  ...   
3       na          aiss@moe.edu.sg  CANBERRA MRT, YISHUN MRT  ...   
4       na    AITONG_SCH@MOE.EDU.SG                Bishan MRT  ...   

    nature_code    session_code mainlevel_code sap_ind autonomous_ind  \
0  CO-ED SCHOOL        FULL DAY        PRIMARY      No             No   
1  CO-ED SCHOOL  SINGLE SESSION      SECONDARY      No             No   
2  CO-ED SCHOOL  SINGLE SESSION        PRIMARY      No             No   
3  CO-ED SCHOOL  SINGLE SESSION      SECONDARY      No             No   
4  CO-ED SCHOOL  SINGLE SESSION        PRIMARY     Yes             No   

  gifted_ind ip_ind mothertongue1_code mothertongue2_code mothertongue3_code  
0         No     No            Chinese              Malay              Tamil  
1         No     No            Chinese              Malay              Tamil  
2         No     No            Chinese              Malay              Tamil  
3         No     No            Chinese              Malay              Tamil  
4         No     No            Chinese                 na                 na  

[5 rows x 31 columns]

In [49]:
# Making a list from the dataframe column 'school_name'
all_school = list(school_data['school_name'])

# Finding out the number of schools based on number of rows (len function)
print('No. of school:', len(all_school))

No. of school: 346


In [50]:
#Converting x to string value
# all_school = ([str(x) for x in all_school])

In [51]:
# Applying the find_postal function which as per mentioned above, scrap for latitude and logitude values 
# based on input, in this case - postal code of school.

find_postal(all_school, 'school_coordinates')

0 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=ADMIRALTY PRIMARY SCHOOL
1 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=ADMIRALTY SECONDARY SCHOOL
2 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=AHMAD IBRAHIM PRIMARY SCHOOL
3 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=AHMAD IBRAHIM SECONDARY SCHOOL
4 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=AI TONG SCHOOL
5 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=ALEXANDRA PRIMARY SCHOOL
6 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=ANCHOR GREEN PRIMARY SCHOOL
7 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=ANDERSON PRIMARY SCHOOL
8 https://de

66 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=CHRIST CHURCH SECONDARY SCHOOL
67 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=CHUA CHU KANG PRIMARY SCHOOL
68 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=CHUA CHU KANG SECONDARY SCHOOL
69 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=CHUNG CHENG HIGH SCHOOL (MAIN)
70 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=CHUNG CHENG HIGH SCHOOL (YISHUN)
71 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=CLEMENTI PRIMARY SCHOOL
72 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=CLEMENTI TOWN SECONDARY SCHOOL
73 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVa

132 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=HOUGANG SECONDARY SCHOOL
133 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=HUA YI SECONDARY SCHOOL
134 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=HUAMIN PRIMARY SCHOOL
135 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=HWA CHONG INSTITUTION
136 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=INNOVA PRIMARY SCHOOL
137 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=JIEMIN PRIMARY SCHOOL
138 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=JING SHAN PRIMARY SCHOOL
139 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=JUNYUAN PRIMARY SCHOOL
140 https://dev

198 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=NORTHOAKS PRIMARY SCHOOL
199 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=NORTHSHORE PRIMARY SCHOOL
200 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=NUS HIGH SCHOOL OF MATHEMATICS AND SCIENCE
201 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=OASIS PRIMARY SCHOOL
202 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=OPERA ESTATE PRIMARY SCHOOL
203 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=ORCHID PARK SECONDARY SCHOOL
204 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=OUTRAM SECONDARY SCHOOL
205 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=PALM

265 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=ST. ANDREW'S JUNIOR COLLEGE
266 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=ST. ANDREW'S JUNIOR SCHOOL
267 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=ST. ANDREW'S SECONDARY SCHOOL
268 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=ST. ANTHONY'S CANOSSIAN PRIMARY SCHOOL
269 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=ST. ANTHONY'S CANOSSIAN SECONDARY SCHOOL
270 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=ST. ANTHONY'S PRIMARY SCHOOL
271 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=ST. GABRIEL'S PRIMARY SCHOOL
272 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails

330 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=YISHUN INNOVA JUNIOR COLLEGE
331 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=YISHUN PRIMARY SCHOOL
332 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=YISHUN SECONDARY SCHOOL
333 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=YISHUN TOWN SECONDARY SCHOOL
334 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=YU NENG PRIMARY SCHOOL
335 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=YUAN CHING SECONDARY SCHOOL
336 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=YUHUA PRIMARY SCHOOL
337 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=YUHUA SECONDARY SCHOOL
3

In [52]:
# The file will be created as school_coordinates.csv
school = pd.read_csv('school_coordinates.csv')
school = school[['BUILDING','LATITUDE', 'LONGTITUDE']]
school.head()

BUILDING  LATITUDE  LONGTITUDE
0  THE LEARNING HARBOUR @ ADMIRALTY PRIMARY SCHOOL  1.442550  103.800214
1                         ADMIRALTY PRIMARY SCHOOL  1.442680  103.800114
2                       ADMIRALTY SECONDARY SCHOOL  1.445891  103.802398
3                     AHMAD IBRAHIM PRIMARY SCHOOL  1.433681  103.832924
4                   AHMAD IBRAHIM SECONDARY SCHOOL  1.436235  103.829986

In [53]:
# Finding out the number of schools based on number of rows (len function)
# To see if there is any difference in the number of rows computed through the API and compare it with the passing input value
print('No. of schools:', len(school))

No. of schools: 439


In [54]:
# Removing duplicates based on address and check the number of rows
school = school.drop_duplicates(subset = ["BUILDING"])
print('No. of school:', len(school))
school

No. of school: 409


BUILDING  LATITUDE  LONGTITUDE
0      THE LEARNING HARBOUR @ ADMIRALTY PRIMARY SCHOOL  1.442550  103.800214
1                             ADMIRALTY PRIMARY SCHOOL  1.442680  103.800114
2                           ADMIRALTY SECONDARY SCHOOL  1.445891  103.802398
3                         AHMAD IBRAHIM PRIMARY SCHOOL  1.433681  103.832924
4                       AHMAD IBRAHIM SECONDARY SCHOOL  1.436235  103.829986
..                                                 ...       ...         ...
434                            ZHENGHUA PRIMARY SCHOOL  1.379802  103.769171
435                          ZHENGHUA SECONDARY SCHOOL  1.388197  103.765252
436                            ZHONGHUA PRIMARY SCHOOL  1.360167  103.869696
437  LEARNING STUDIO BY ZENITANT (ZHONGHUA PRIMARY ...  1.360203  103.869762
438                          ZHONGHUA SECONDARY SCHOOL  1.349097  103.869800

[409 rows x 3 columns]

In [55]:
# Applying the find_nearest() function to compute the nearest supermarket to flat and find the number of supermarkets within 2km radius.
# Passing the values after applying the find_nearest function into a dataframe with columns as shown below, in such order.

nearest_school = find_nearest(hdb, school)
flat_school = pd.DataFrame.from_dict(nearest_school).T
flat_school = flat_school.rename(columns={0: 'flat', 1: 'school', 2: 'school_dist', 3: 'num_school_2km'}).reset_index().drop('index', axis=1)
flat_school.head()

flat                     school school_dist num_school_2km
0  401 ANG MO KIO AVE 10  TOWNSVILLE PRIMARY SCHOOL    0.159639             27
1  414 ANG MO KIO AVE 10   TECK GHEE PRIMARY SCHOOL    0.300109             23
2  438 ANG MO KIO AVE 10      DEYI SECONDARY SCHOOL    0.212322             23
3  418 ANG MO KIO AVE 10   TECK GHEE PRIMARY SCHOOL    0.338325             23
4  443 ANG MO KIO AVE 10      DEYI SECONDARY SCHOOL      0.4984             23

In [56]:
#Saving the dataframe into a csv file
flat_school.to_csv(r'flat_school.csv')

### No requirement for scraping of park locations, only required to apply find_nearest function

Below file 'parks-geojson.csv' is taken from Data.gov website below.

https://data.gov.sg/dataset/nparks-parks?resource_id=b7318f05-544c-4eb1-9a62-1142cd70d28c

Like hawker centres, the longitude and latitude values has already been tabulated.
We only require to apply the find_nearest function to compute the number of parks and the nearest parks to every HDB flat.

In [57]:
# Reading file and convert to a dataframe
park_data = pd.read_csv('parks-geojson.csv')
park_data.head()

Name  latitude   longitude  altitude geometry  \
0  kml_1  1.346175  103.960857         0    Point   
1  kml_2  1.389904  103.978979         0    Point   
2  kml_3  1.409913  103.923131         0    Point   
3  kml_4  1.379237  103.866967         0    Point   
4  kml_5  1.372761  103.829073         0    Point   

                                         Description  
0  <center><table><tr><th colspan='2' align='cent...  
1  <center><table><tr><th colspan='2' align='cent...  
2  <center><table><tr><th colspan='2' align='cent...  
3  <center><table><tr><th colspan='2' align='cent...  
4  <center><table><tr><th colspan='2' align='cent...

In [58]:
#Drop unwanted columns to have only Name, latitude and longitude
park_data.drop(park_data.columns.difference(['Name','latitude', 'longitude']), 1, inplace=True)
park_data

Name  latitude   longitude
0      kml_1  1.346175  103.960857
1      kml_2  1.389904  103.978979
2      kml_3  1.409913  103.923131
3      kml_4  1.379237  103.866967
4      kml_5  1.372761  103.829073
..       ...       ...         ...
345  kml_346  1.367964  103.890365
346  kml_347  1.390172  103.989190
347  kml_348  1.355196  103.885955
348  kml_349  1.391640  103.903736
349  kml_350  1.325534  103.947490

[350 rows x 3 columns]

In [59]:
# Applying the find_postal function which as per mentioned above, scrap for latitude and logitude values 
# based on input, in this case - postal code of park.
# The file will be created as park_coordinates.csv
nearest_park = find_nearest(hdb, park_data)
flat_park = pd.DataFrame.from_dict(nearest_park).T
flat_park = flat_park.rename(columns={0: 'flat', 1: 'park', 2: 'park_dist', 3: 'num_park_2km'}).reset_index().drop(['index','park'], axis=1)
flat_park.head()

flat park_dist num_park_2km
0  401 ANG MO KIO AVE 10  0.762607           18
1  414 ANG MO KIO AVE 10  0.747256           18
2  438 ANG MO KIO AVE 10  0.558461           17
3  418 ANG MO KIO AVE 10  0.751536           18
4  443 ANG MO KIO AVE 10  0.842097           15

In [60]:
# Save above data into csv file
flat_park.to_csv(r'flat_park.csv')

### Applying the dist_from_location() function
As the distance to City Hall is one of the features that will be considered for machine learning later, we tabulate to find distance to City Hall for every HDB flat as per below.


In [61]:
flat_coord = pd.read_csv('flat_coordinates.csv')
#Selecting only the address, latitude and longitude columns of flat_coord dataframe
flat_coord = flat_coord[['address','latitude','longitude']]

#With the 3 parameters (address, latitude and longitude), we use it to apply the dist_from_location to get the distance of every flat to City Hall.
# latitude and longitude coordinates are passed in location 
dist_city_hall = dist_from_location(flat_coord, (1.292936, 103.852585))
dist_city_hall = pd.DataFrame.from_dict(dist_city_hall).T
dist_city_hall = dist_city_hall.rename(columns={0: 'flat', 1: 'dist_city_hall'}).reset_index().drop(['index'], axis=1)
dist_city_hall.head()

flat dist_city_hall
0  401 ANG MO KIO AVE 10       7.543853
1  414 ANG MO KIO AVE 10        7.86668
2  438 ANG MO KIO AVE 10       8.187729
3  418 ANG MO KIO AVE 10       7.963013
4  443 ANG MO KIO AVE 10        8.05594

In [65]:
# Reading file and convert to a dataframe
dist_city_hall.to_csv(r'dist_city_hall.csv')


### Merging all amenities datasets into 1 dataframe

After we have computed all the amenities data in getting the number of amenities found within 2km of HDB flat and find the nearest amenities to every HDB flat via the find_nearest function, we can merge them into one file.


In [63]:
## Merging all dataset via joining on outer with the column 'flat' as common value in all dataframes.

flat_amenities = flat_school.merge(flat_hawker, on='flat', how='outer')
flat_amenities = flat_amenities.merge(flat_park, on='flat', how='outer')
flat_amenities = flat_amenities.merge(flat_mall, on='flat', how='outer')
flat_amenities = flat_amenities.merge(flat_mrt, on='flat', how='outer')
flat_amenities = flat_amenities.merge(flat_supermarket, on='flat', how='outer')
flat_amenities = flat_amenities.merge(dist_city_hall, on='flat', how='outer')
flat_amenities.head()



flat                     school school_dist  \
0  401 ANG MO KIO AVE 10  TOWNSVILLE PRIMARY SCHOOL    0.159639   
1  414 ANG MO KIO AVE 10   TECK GHEE PRIMARY SCHOOL    0.300109   
2  438 ANG MO KIO AVE 10      DEYI SECONDARY SCHOOL    0.212322   
3  418 ANG MO KIO AVE 10   TECK GHEE PRIMARY SCHOOL    0.338325   
4  443 ANG MO KIO AVE 10      DEYI SECONDARY SCHOOL      0.4984   

  num_school_2km                                             hawker  \
0             27       Ang Mo Kio Ave 10 Blk 409 (Teck Ghee Square)   
1             23       Ang Mo Kio Ave 10 Blk 409 (Teck Ghee Square)   
2             23  Ang Mo Kio Ave 10 Blk 453A (Chong Boon Market ...   
3             23       Ang Mo Kio Ave 10 Blk 409 (Teck Ghee Square)   
4             23  Ang Mo Kio Ave 10 Blk 453A (Chong Boon Market ...   

  hawker_dist num_hawker_2km park_dist num_park_2km  \
0    0.184587              9  0.762607           18   
1    0.254548             10  0.747256           18   
2    0.315933             10  0.558461           17   
3     0.28037             10  0.751536           18   
4    0.281704              9  0.842097           15   

                            mall mall_dist num_mall_2km              MRT  \
0  myVillage At Serangoon Garden  1.161261            5  Ang Mo Kio NS16   
1                        AMK Hub   0.80033            5  Ang Mo Kio NS16   
2                        AMK Hub   0.64598            5  Ang Mo Kio NS16   
3                        AMK Hub   0.78492            5  Ang Mo Kio NS16   
4                        AMK Hub  0.936753            5  Ang Mo Kio NS16   

   MRT_dist num_MRT_2km supermarket_name supermarket_dist num_supermarket_2km  \
0  1.174407           4           560421         0.546651                  17   
1  0.781515           4           560421         0.155975                  17   
2  0.575116           4           560421          0.19899                  15   
3  0.745965           4           560421         0.131461                  17   
4  0.865839           4           560421         0.351198                  16   

  dist_city_hall  
0       7.543853  
1        7.86668  
2       8.187729  
3       7.963013  
4        8.05594

In [64]:
#Saving the dataframe into a csv file
flat_amenities.to_csv(r'flat_amenities.csv')